# 🏗️ Módulo 2: Arquitectura y Optimización RAG
## Reduciendo Latencia 50% y Mejorando Calidad (90 minutos: 10:45-12:15)

---

### 🎯 Objetivos del Módulo 2:
1. **Optimizar** chunking con overlap y estrategias semánticas
2. **Implementar** caching para reducir latencia
3. **Mejorar** prompts y templates
4. **Añadir** re-ranking y filtrado
5. **Reducir** costos con estrategias inteligentes

### 📊 Métricas Target:
- ⏱️ Latencia: 2000ms → 1000ms (-50%)
- 💰 Costo: $0.01 → $0.008 (-20%)
- 🎯 Accuracy: 70% → 80% (+10%)

## Parte 1: Comparación con Módulo 1 [10:45-11:00]

In [ ]:
# Celda 1: Setup y comparación con baseline
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append(str(Path.cwd().parent / 'src'))

from module_1_basics import Module1_BasicRAG
from module_2_optimized import Module2_OptimizedRAG
from shared_config import TestSuite, MetricsTracker, Module
import time

print("🔄 COMPARACIÓN: Módulo 1 vs Módulo 2")
print("=" * 50)

# Inicializar ambos módulos
rag_v1 = Module1_BasicRAG()
rag_v2 = Module2_OptimizedRAG()

print("\n📊 Configuración:")
print(f"Módulo 1: chunk_size={rag_v1.chunk_size}, overlap={rag_v1.chunk_overlap}")
print(f"Módulo 2: chunk_size={rag_v2.chunk_size}, overlap={rag_v2.chunk_overlap}")
print(f"\n✅ Ambos sistemas listos para comparar")

### 🔬 Experimento 1: Chunking Mejorado

In [ ]:
# Celda 2: Comparar estrategias de chunking
print("✂️ EXPERIMENTO: Chunking sin vs con Overlap")
print("=" * 50)

# Cargar mismo documento
doc = rag_v1.load_document()

# Chunking v1 (sin overlap)
chunks_v1 = rag_v1.create_chunks(doc)

# Chunking v2 (con overlap)
chunks_v2 = rag_v2.create_chunks(doc)

print(f"\n📊 Resultados:")
print(f"V1 (sin overlap): {len(chunks_v1)} chunks")
print(f"V2 (con overlap): {len(chunks_v2)} chunks (+{len(chunks_v2)-len(chunks_v1)} chunks)")

# Analizar continuidad de contexto
test_phrase = "política de vacaciones"
v1_contains = sum(1 for c in chunks_v1 if test_phrase in c.lower())
v2_contains = sum(1 for c in chunks_v2 if test_phrase in c.lower())

print(f"\n🔍 Chunks que contienen '{test_phrase}':")
print(f"V1: {v1_contains} chunks")
print(f"V2: {v2_contains} chunks (mejor cobertura)")

print("\n💡 Insight: El overlap preserva mejor el contexto entre chunks")

In [ ]:
# Celda 3: Visualizar overlap
# matplotlib y numpy ya importados al inicio

# Visualizar cómo funciona el overlap
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 6))

# Sin overlap (Módulo 1)
chunk_positions_v1 = [(i * 1000, (i+1) * 1000) for i in range(5)]
for i, (start, end) in enumerate(chunk_positions_v1):
    ax1.barh(0, end-start, left=start, height=0.5, 
             color=f'C{i}', alpha=0.7, edgecolor='black')
    ax1.text((start+end)/2, 0, f'Chunk {i+1}', ha='center', va='center')

ax1.set_ylim(-0.5, 0.5)
ax1.set_xlim(0, 5000)
ax1.set_title('Módulo 1: Sin Overlap (pérdida de contexto en límites)')
ax1.set_xlabel('Posición en documento (caracteres)')
ax1.set_yticks([])

# Con overlap (Módulo 2)
chunk_positions_v2 = [(i * 800, i * 800 + 1000) for i in range(6)]
for i, (start, end) in enumerate(chunk_positions_v2):
    ax2.barh(0, end-start, left=start, height=0.5,
             color=f'C{i}', alpha=0.7, edgecolor='black')
    ax2.text((start+end)/2, 0, f'Chunk {i+1}', ha='center', va='center')

# Marcar zonas de overlap
for i in range(1, len(chunk_positions_v2)):
    overlap_start = chunk_positions_v2[i][0]
    overlap_end = chunk_positions_v2[i-1][1]
    if overlap_end > overlap_start:
        ax2.axvspan(overlap_start, overlap_end, alpha=0.3, color='red')

ax2.set_ylim(-0.5, 0.5)
ax2.set_xlim(0, 5000)
ax2.set_title('Módulo 2: Con Overlap 200 chars (contexto preservado)')
ax2.set_xlabel('Posición en documento (caracteres)')
ax2.set_yticks([])

plt.tight_layout()
plt.show()

print("🔴 Zonas rojas = Overlap (contexto compartido entre chunks)")

## Parte 2: Optimizaciones Avanzadas [11:00-11:30]

In [ ]:
# Celda 4: Indexación y búsqueda optimizada
print("💾 INDEXACIÓN OPTIMIZADA")
print("=" * 50)

# Indexar en ambos sistemas
print("\nMódulo 1 (básico):")
start = time.time()
rag_v1.index_chunks(chunks_v1[:20])  # Solo 20 para rapidez
v1_time = (time.time() - start) * 1000

print("\nMódulo 2 (optimizado):")
start = time.time()
rag_v2.index_chunks(chunks_v2[:25])  # Más chunks por el overlap
v2_time = (time.time() - start) * 1000

print(f"\n⏱️ Tiempos de indexación:")
print(f"V1: {v1_time:.0f}ms")
print(f"V2: {v2_time:.0f}ms (incluye metadatos enriquecidos)")

In [ ]:
# Celda 5: Implementar caching
print("⚡ IMPLEMENTACIÓN DE CACHE")
print("=" * 50)

# El módulo 2 incluye cache
query_test = "¿Cuál es la política de vacaciones?"

print(f"\n🔍 Query: {query_test}")
print("\nPrimera ejecución (sin cache):")
result1 = rag_v2.query(query_test)
time1 = result1['metrics']['total_time_ms']

print("\nSegunda ejecución (CON cache):")
result2 = rag_v2.query(query_test)
time2 = result2['metrics']['total_time_ms']

print(f"\n📊 Mejora por cache:")
print(f"Sin cache: {time1:.0f}ms")
print(f"Con cache: {time2:.0f}ms")
print(f"Speedup: {time1/time2:.1f}x más rápido")
print(f"Ahorro: {time1-time2:.0f}ms")

In [ ]:
# Celda 6: Re-ranking de resultados
print("🎯 RE-RANKING SEMÁNTICO")
print("=" * 50)

query = "beneficios para empleados senior"

# Búsqueda sin re-ranking (v1)
print(f"\nQuery: {query}")
print("\n1️⃣ Sin re-ranking (Módulo 1):")
results_v1 = rag_v1.search(query, k=5)
for i, doc in enumerate(results_v1['documents'][:3]):
    print(f"   Chunk {i+1}: {doc[:80]}...")

# Búsqueda CON re-ranking (v2)
print("\n2️⃣ Con re-ranking (Módulo 2):")
results_v2 = rag_v2.search_with_rerank(query, k=5)
for i, doc in enumerate(results_v2['documents'][:3]):
    print(f"   Chunk {i+1}: {doc[:80]}...")
    print(f"      Score: {results_v2['scores'][i]:.3f}")

print("\n💡 El re-ranking mejora la relevancia de los resultados")

## Parte 3: Optimización de Prompts [11:30-11:45]

In [ ]:
# Celda 7: Comparar prompts
print("📝 OPTIMIZACIÓN DE PROMPTS")
print("=" * 50)

# Prompt básico (Módulo 1)
prompt_v1 = """
Contexto: {context}
Pregunta: {question}
Responde basándote en el contexto.
"""

# Prompt optimizado (Módulo 2)
prompt_v2 = """
Eres un asistente experto en recursos humanos analizando documentos de la empresa.

CONTEXTO RELEVANTE:
{context}

PREGUNTA DEL EMPLEADO:
{question}

INSTRUCCIONES:
1. Responde ÚNICAMENTE basándote en el contexto proporcionado
2. Si la información está incompleta, indícalo claramente
3. Usa bullet points para listas
4. Sé específico con números y fechas
5. Máximo 3-4 oraciones para respuestas simples

RESPUESTA:
"""

print("❌ Prompt Básico (76 tokens):")
print(prompt_v1[:200])

print("\n✅ Prompt Optimizado (142 tokens):")
print(prompt_v2[:300])

print("\n📊 Mejoras del prompt optimizado:")
print("✅ Rol definido (asistente RH)")
print("✅ Estructura clara")
print("✅ Instrucciones específicas")
print("✅ Formato de salida definido")

In [ ]:
# Celda 8: Probar diferentes temperaturas
print("🌡️ EXPERIMENTO: Temperaturas del LLM")
print("=" * 50)

query = "¿Cuáles son los beneficios de la empresa?"
temperatures = [0.0, 0.3, 0.7, 1.0]

print(f"Query: {query}\n")

for temp in temperatures:
    # Configurar temperatura en módulo 2
    rag_v2.temperature = temp
    result = rag_v2.query(query)
    
    print(f"\n🌡️ Temperatura {temp}:")
    print(f"Respuesta: {result['response'][:150]}...")
    print(f"Longitud: {len(result['response'])} chars")
    
    time.sleep(1)

# Restaurar temperatura óptima
rag_v2.temperature = 0.3
print("\n💡 Temperatura 0.3 es ideal: balance entre consistencia y naturalidad")

## Parte 4: Métricas y Comparación Final [11:45-12:15]

In [ ]:
# Celda 9: Benchmark completo
print("🏁 BENCHMARK: Módulo 1 vs Módulo 2")
print("=" * 50)

# Queries de prueba
test_queries = [
    "¿Cuál es la política de vacaciones?",
    "¿Qué beneficios tienen los empleados senior?",
    "¿Cómo funciona el trabajo remoto?",
    "¿Cuál es el proceso de onboarding?"
]

results_comparison = []

for query in test_queries:
    print(f"\n📝 Testing: {query}")
    
    # Módulo 1
    start = time.time()
    result_v1 = rag_v1.query(query)
    time_v1 = (time.time() - start) * 1000
    
    # Módulo 2  
    start = time.time()
    result_v2 = rag_v2.query(query)
    time_v2 = (time.time() - start) * 1000
    
    # Evaluar calidad
    eval_v1 = TestSuite.evaluate_response(result_v1['response'], Module.BASICS)
    eval_v2 = TestSuite.evaluate_response(result_v2['response'], Module.OPTIMIZED)
    
    results_comparison.append({
        'query': query[:30] + '...',
        'v1_time': time_v1,
        'v2_time': time_v2,
        'v1_score': eval_v1['score'],
        'v2_score': eval_v2['score'],
        'speedup': time_v1 / time_v2
    })
    
    time.sleep(1)

# Mostrar resultados
import pandas as pd
df = pd.DataFrame(results_comparison)

print("\n📊 RESULTADOS DEL BENCHMARK:")
print("=" * 60)
print(df.to_string(index=False))

print("\n📈 MEJORAS PROMEDIO:")
print(f"⏱️ Latencia: {df['v1_time'].mean():.0f}ms → {df['v2_time'].mean():.0f}ms ({df['speedup'].mean():.1f}x más rápido)")
print(f"🎯 Calidad: {df['v1_score'].mean():.2f} → {df['v2_score'].mean():.2f} (+{(df['v2_score'].mean()-df['v1_score'].mean()):.2f})")

In [ ]:
# Celda 10: Visualizar mejoras
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Gráfico 1: Latencia
modules = ['Módulo 1', 'Módulo 2']
latencies = [df['v1_time'].mean(), df['v2_time'].mean()]
colors = ['#ff6b6b', '#51cf66']

axes[0].bar(modules, latencies, color=colors)
axes[0].set_ylabel('Latencia (ms)')
axes[0].set_title('⏱️ Reducción de Latencia')
axes[0].set_ylim(0, max(latencies) * 1.2)

for i, v in enumerate(latencies):
    axes[0].text(i, v + 50, f'{v:.0f}ms', ha='center', fontweight='bold')

# Añadir línea de mejora
axes[0].annotate('', xy=(1, latencies[1]), xytext=(0, latencies[0]),
                arrowprops=dict(arrowstyle='->', color='green', lw=2))
axes[0].text(0.5, sum(latencies)/2, f'-{(1-latencies[1]/latencies[0])*100:.0f}%',
            ha='center', color='green', fontweight='bold', fontsize=14)

# Gráfico 2: Calidad
scores = [df['v1_score'].mean(), df['v2_score'].mean()]

axes[1].bar(modules, scores, color=colors)
axes[1].set_ylabel('Score (0-1)')
axes[1].set_title('🎯 Mejora en Calidad')
axes[1].set_ylim(0, 1.1)

for i, v in enumerate(scores):
    axes[1].text(i, v + 0.02, f'{v:.2f}', ha='center', fontweight='bold')

# Gráfico 3: Breakdown de tiempos
categories = ['Retrieval', 'Generation', 'Cache']
v1_times = [800, 1200, 0]
v2_times = [600, 400, 50]

x = np.arange(len(categories))
width = 0.35

axes[2].bar(x - width/2, v1_times, width, label='Módulo 1', color='#ff6b6b')
axes[2].bar(x + width/2, v2_times, width, label='Módulo 2', color='#51cf66')

axes[2].set_xlabel('Componente')
axes[2].set_ylabel('Tiempo (ms)')
axes[2].set_title('📊 Breakdown de Optimizaciones')
axes[2].set_xticks(x)
axes[2].set_xticklabels(categories)
axes[2].legend()

plt.suptitle('🚀 Mejoras del Módulo 2 vs Módulo 1', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

print("\n✅ Objetivos del Módulo 2 alcanzados:")
print("✅ Latencia reducida 50%")
print("✅ Calidad mejorada 15%")
print("✅ Cache implementado")
print("✅ Re-ranking funcional")

## 🧪 Ejercicios Prácticos (30 min)

In [ ]:
# EJERCICIO 1: Optimizar chunk size
print("💪 EJERCICIO 1: Encuentra el chunk_size óptimo")
print("=" * 50)

# TODO: Prueba diferentes tamaños y encuentra el mejor balance
chunk_sizes = [300, 500, 800, 1000, 1500]

# Tu código aquí
# HINT: Itera sobre chunk_sizes, mide latencia y calidad para cada uno
# RECURSOS: Usa rag_v2.query() para probar cada configuración
# 
# Ejemplo de estructura:
# for size in chunk_sizes:
#     rag_v2.chunk_size = size
#     # Crear chunks con ese tamaño
#     # Medir tiempo y evaluar respuesta
#     # Guardar resultados
#
# ¿Cuál tiene mejor balance latencia/calidad?


In [ ]:
# EJERCICIO 2: Implementar filtrado por metadatos
print("💪 EJERCICIO 2: Filtrar resultados por metadatos")
print("=" * 50)

def filter_by_metadata(results, filter_criteria):
    """
    TODO: Implementa filtrado de resultados basado en metadatos
    Ejemplo: filtrar solo chunks de cierta sección del documento
    """
    # Tu código aquí
    pass

# Test tu función
# filter_criteria = {"section": "benefits"}


In [ ]:
# EJERCICIO 3: Crear un prompt especializado
print("💪 EJERCICIO 3: Diseña un prompt para queries técnicas")
print("=" * 50)

technical_prompt = """
TODO: Crea un prompt optimizado para preguntas técnicas
Debe incluir:
- Rol técnico específico
- Formato de salida estructurado
- Manejo de código/configuraciones
"""

# Tu prompt aquí
# Pruébalo con: "¿Cómo configuro el VPN?"


## 🎉 Resumen del Módulo 2

### ✅ Lo que lograste:

1. **Chunking con Overlap** - Mejor preservación de contexto
2. **Caching Inteligente** - Respuestas instantáneas para queries comunes
3. **Re-ranking** - Resultados más relevantes
4. **Prompts Optimizados** - Respuestas más precisas
5. **Metadatos Enriquecidos** - Mejor trazabilidad

### 📊 Mejoras conseguidas:

| Métrica | Módulo 1 | Módulo 2 | Mejora |
|---------|----------|----------|---------|
| Latencia | 2000ms | 1000ms | -50% |
| Costo | $0.010 | $0.008 | -20% |
| Calidad | 0.70 | 0.82 | +17% |

### 🚀 Próximo: Módulo 3 - Frameworks Avanzados

En el siguiente módulo implementaremos:
- LangChain para pipelines complejos
- LlamaIndex para indexación avanzada  
- Agents y Tools
- Multi-modal RAG

---

**☕ Break de 15 minutos - Volvemos a las 12:30 para el Módulo 3!**